The GamsStucture package is a package I (Mitch) am developing specifically
to bridge the gap between GAMS and Julia. It's incredibly useful for building
vectorized models.

In [1]:
using JuMP
using Clp

using GamsStructure

This is where I initialize all the data. Define sets and parameters and set parameter values.

In [2]:
GU = GamsUniverse()

@GamsSet(GU,:j,"Products",begin
    lager, ""
    ale, ""
end)

@GamsSet(GU,:i,"Ingredients",begin
    malt, "Malt"
    yeast, "Yeast"
    dehops, "Germand Hops"
    wihops, "Wisconsin Hops"
end)


@GamsParameters(GU,begin
    :p, (:j,), "Profit by product"
    :s, (:i,), "Supply by ingredient"
    :a, (:i,:j), "Requirements"
end)


GU[:p][[:lager]] = 120
GU[:p][[:ale]] = 90

GU[:s][[:malt]] = 4800
GU[:s][[:yeast]] = 1750
GU[:s][[:dehops]] = 1000
GU[:s][[:wihops]] = 1750

GU[:a][[:malt],[:lager]] = 4
GU[:a][[:yeast],[:lager]] = 1
GU[:a][[:dehops],[:lager]] = 1
GU[:a][[:wihops],[:lager]] = 0

GU[:a][[:malt],[:ale]] = 2
GU[:a][[:yeast],[:ale]] = 1
GU[:a][[:dehops],[:ale]] = 0
GU[:a][[:wihops],[:ale]] = 1



1

In [3]:
primal = Model(Clp.Optimizer)

I = [i for i∈GU[:i]] #You tend to type the set names very often in models. 
J = [e for e∈GU[:j]] #pull out the symbols from the set j

"""
1. Here the variables macro is over kill since there is only one variable. Habit
2. This is actually creating two variables, one for each element of J. You can
   set bounds and even have the bounds come from other vectors.
"""

@variables(primal,begin
    Y[J]>=0 #Production levels
end)

"""
.* means coordinate wise multiplication, simplifies the syntax. I could have 
typed:

sum(GU[:p][[j]]*Y[j] for j∈J)

To get the ∈ I typed \\in (except with a single backslash) and pressed tab. 

"∈" is the same as "=" is the same as "in" in a loop
"""

@objective(primal, Max, sum(GU[:p][J].*Y[J])) 


"""
1. This is how you define a vectorized constraint. After the constraint name
   include, in brackets, the variable. In this case it's vectorized over I
   and I'm going to use these as the name i

2. Be careful with GamsParameters, if you want a single value you need more 
   brakets. So GU[:s][i] will give an error since i isn't a valid set in GU.
   Each coordinate of GU[:s] is either a GamsSet, Symbol (like :i) or a list
   of symbols. Examples below.
"""

@constraint(primal, supply[i = I], 
    sum(GU[:a][[i],J].*Y[J]) <= GU[:s][[i]] 
)

optimize!(primal)

solution_summary(primal)


Coin0506I Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
Clp0006I 0  Obj -0 Dual inf 210 (2)
Clp0006I 2  Obj 177000
Clp0000I Optimal - objective value 177000
Coin0511I After Postsolve, objective 177000, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 177000 - 2 iterations time 0.002, Presolve 0.00


* Solver : Clp

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "0 - optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.77000e+05

* Work counters
  Solve time (sec)   : 0.00000e+00


In [4]:
for var in all_variables(primal)
    println("$var -> $(value(var))")
end

Y[lager] -> 650.0
Y[ale] -> 1100.0


In [5]:
value.(primal[:supply]) #I'm surprised this worked.

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:malt, :yeast, :dehops, :wihops]
And data, a 4-element Vector{Float64}:
 4800.0
 1750.0
  650.0
 1100.0

# Parameter Examples

In [6]:
GU[:s] #The entire set, you can see the domain here

Description: Supply by ingredient
Domain: (:i,)



[4800.0, 1750.0, 1000.0, 1750.0]

In [7]:
GU[:s][:i] #Just the array of values

4-element Vector{Float64}:
 4800.0
 1750.0
 1000.0
 1750.0

In [8]:
GU[:s][[:malt]] # A single value

4800.0

In [9]:
GU[:s][[:malt,:yeast]] #Two values

2-element Vector{Float64}:
 4800.0
 1750.0

# Primal Model

In [10]:
dual = Model(Clp.Optimizer)

I = [i for i∈GU[:i]]
J = [j for j∈GU[:j]]

@variables(dual,begin
    PI[I]>=0 #shadow price of ingredient i
end)


@objective(dual, Min, sum(PI[I].*GU[:s][I]))

@constraint(dual, profit[j=J], sum(PI[I].*GU[:a][I,[j]]) >= GU[:p][[j]])

optimize!(dual)

solution_summary(dual)

Coin0506I Presolve 2 (0) rows, 4 (0) columns and 6 (0) elements
Clp0006I 0  Obj 0 Primal inf 75 (2)
Clp0006I 2  Obj 177000
Clp0000I Optimal - objective value 177000
Clp0032I Optimal objective 177000 - 2 iterations time 0.002


* Solver : Clp

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "0 - optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.77000e+05

* Work counters
  Solve time (sec)   : 0.00000e+00


In [11]:
for var ∈ all_variables(dual)
    println("$var -> $(value(var))")
end

PI[malt] -> 15.0
PI[yeast] -> 60.0
PI[dehops] -> 0.0
PI[wihops] -> 0.0


In [12]:
value.(dual[:profit]) 

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:lager, :ale]
And data, a 2-element Vector{Float64}:
 120.0
  90.0